### Obtain Minio endpoint URL programatically

In [ ]:
from kubernetes import client as k8s_client
from kubernetes import config as k8s_config
from kubeflow.fairing.utils import is_running_in_k8s
from kubernetes.client.rest import ApiException
import logging

namespace = fairing_utils.get_current_k8s_namespace()
if is_running_in_k8s():
    k8s_config.load_incluster_config()
else:
    k8s_config.load_kube_config()

api_client = k8s_client.CoreV1Api()
minio_service_endpoint = None
try:
    minio_service_endpoint = api_client.read_namespaced_service(name='minio-service', namespace='kubeflow')
except ApiException as e:
    if e.status == 403:
        logging.warning(
            f"The service account doesn't have sufficient privileges "
            f"to get the kubeflow minio-service. "
            f"You will have to manually enter the minio cluster-ip. "
            f"To make this function work ask someone with cluster "
            f"priveleges to create an appropriate "
            f"clusterrolebinding by running a command.\n"
            f"kubectl create --namespace=kubeflow rolebinding "
            f"--clusterrole=kubeflow-view "
            f"--serviceaccount={namespace}:default-editor "
            f"{namespace}-minio-view")
        logging.error(f"API access denied with reason: {e.reason}")

s3_endpoint_ip = minio_service_endpoint.spec.cluster_ip
s3_endpoint_port = minio_service_endpoint.spec.ports[0].port
s3_endpoint = f"{s3_endpoint_ip}:{s3_endpoint_port}"
minio_endpoint = f"http://{s3_endpoint}"

logging.info(f"Running in namespace {namespace}")
logging.info(f"Using minio instance with endpoint '{minio_endpoint}'")


### Parameters needed for a Minio client session

In [ ]:
if minio_endpoint == "http://:":
    minio_endpoint = "http://minio-service.kubeflow.svc.cluster.local:9000"
minio_username = "minio"
minio_key = "minio123"
minio_region = "us-east-1"

### Setup a Minio client session

In [ ]:
import boto3
from botocore.client import Config

session = boto3.session.Session()
s3 = boto3.client('s3',
                   endpoint_url=minio_endpoint,
                   aws_access_key_id=minio_username,
                   aws_secret_access_key=minio_key,
                   config=Config(signature_version='s3v4'),
                   region_name=minio_region,
                   use_ssl=False)


### List a Minio bucket

In [ ]:
response = s3.list_buckets()

# Check the existence of the given bucket
mybucket='pcdas'
found = False
print(f'Checking the existence of my bucket {mybucket}:')
for bucket in response['Buckets']:
    if mybucket == bucket["Name"]:
        print(f'  {bucket["Name"]} {bucket["CreationDate"]} {response["Owner"]["DisplayName"]}')
        found = True
        break
if not found:
    print(f'  {mybucket} is not found.')

### Source of population dataset
 * [United Nations – Dept. of Economic and Social Affairs, population dynamics](https://population.un.org/wpp/Download/Standard/CSV/)
 
 ```
 $ curl "https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv" > TotalPopulation.csv
 $ head -1 TotalPopulation.csv
 LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
 $ gzip TotalPopulation.csv
 $ mc cp TotalPopulation.csv.gz s3/<bucket>/mycsvbucket/sampledata
 ```
 
### Download the previously uploaded file to /tmp directory
 

In [ ]:
file_name ='TotalPopulation.csv.gz'
with open('/tmp/' + file_name, 'wb') as f:
    s3.download_fileobj(mybucket, 'mycsvbucket/sampledata/' + file_name, f)

In [ ]:
!ls -l /tmp/TotalPopulation.csv.gz